# Исследование надежности заемщиков


## Открываем таблицу и изучаем общую информацию о данных

**Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраняем в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

** Выведим первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведим основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведим количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

** Обработаем значения в столбце `days_employed`: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**З Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится вам для исследования.

** Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

** В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.**

In [14]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
data.groupby('children').agg({'debt':['count','sum', lambda x: x.mean()*100]})

debt                 
          count   sum <lambda_0>
children                        
0         14091  1063   7.543822
1          4808   444   9.234609
2          2052   194   9.454191
3           330    27   8.181818
4            41     4   9.756098
5             9     0   0.000000

**Вывод:** 
В целом можно говорить о том, что коливчество детей в семей не влияет на возврат кредита в срок, поскольку данные разнятся незначительно. Однако бездетные семьи возвращают кредит в срок - чуть чаще. Нулевые значения для семей с пятью детьми, скорее всего, связаны, с маленькой выборкой. 

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
# data.groupby('family_status_id').agg({'debt':['count','sum', lambda x: (str(round((x.mean()*100),2)) + '%' )]})

#сделала по family status - так действительно проще.
data.groupby('family_status').agg({'debt':['count','sum', lambda x: (str(round((x.mean()*100),2)) + '%' )]})



debt                
                       count  sum <lambda_0>
family_status                               
Не женат / не замужем   2796  273      9.76%
в разводе               1189   84      7.06%
вдовец / вдова           951   63      6.62%
гражданский брак        4134  385      9.31%
женат / замужем        12261  927      7.56%

**Вывод:** 
Те, кто был или сотоит в барке, реже возвращают кредит в срок. 


**комменатрий студента** 

Те, кто был или состоит в браке, чаще возвращают кредит в срок. То же касается, и вдовцов. Есть предположение, что это связано с тем, что эти люди уже привыкли брать на себя ответмственность не только за себя и свои дела, но и за других людей. Вероятно , они более умеют. управлять финансами и планировать траты. 

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
data.groupby('total_income_category').agg({'debt':['count','sum', lambda x: (str(round((x.mean()*100),2)) + '%' )]})

debt                 
                       count   sum <lambda_0>
total_income_category                        
A                         25     2       8.0%
B                       5014   354      7.06%
C                      15921  1353       8.5%
D                        349    21      6.02%
E                         22     2      9.09%

    **Вывод:** Фактически можем сравнивать только две категории людей с доходом от 30 до 50 тыс. рублей, и от 50 до 200 тысяч рублей, поскольку в других категориях кредитов слишком мало. В этих двух категориях чаще кредит в во время возращают люди с меньшим доходом. 

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [25]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.

data.groupby('purpose_category').agg({'debt':['count','sum', lambda x: (str(round((x.mean()*100),2)) + '%' )]})

debt                
                          count  sum <lambda_0>
purpose_category                               
операции с автомобилем     4279  400      9.35%
операции с недвижимостью  10751  780      7.26%
получение образования      3988  369      9.25%
проведение свадьбы         2313  183      7.91%

**Вывод:** Люди, которые берут кредит на операции с автомобилем и получение образования, реже возврщают кредит в срок. 

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* невнимательность сотрудников, которые вносят данные, опечатки в данных. 

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Медианные значения точнее срених значений отражают количественные данные в рамках одной катогории. Среднее значение гораздо проще и понятно всем, его можно использовать, когда выбросы не имеют значения. То есть ксли мы будем считать среднуюю зарпалату в Москве с учетом самой высокой зп - мы получим значение, далекое от реальных средних зарплат большинства людей, здесь больше подойдет медиана. Если же мы считаем средний доход в рамках одной семьи - нам подойдет среднее значение. 



### Шаг 4: общий вывод.

Напишите ваш общий вывод.

1. В процессе работы мы проверили доли возврата кредита в срок в зависимости от разных категорий заемщиков: семейное положение, наличие и количество детей, уровень дохода, а так же от целей кредита. 

2. В целом проценты невозврата кредита разнятся незначительно - примерно 1-2 пп, при этом значаени чаще всего не достигают 10%. 

3. Проанализировать каждую категорию в полной мере - не представляется возможным, так в некоторых категориях данных, например, заещиков с высоким уровнем дохода выше 1 млн рублей в месяц, а также семей с пятью детьми - слишком мало. 

4. Самыми надежными заемщиками по уровню дохода можно земщиков с доходом от 30 до 50 тысяч рублей в месяц, далее от 50 до 200 тысяч рублей. Однако эти данные вызываю сомнения, и стоит проследить регионы проживаения эти заещиков.Так как уровень дохода в городах-милионниках или небольших населенных пунктах разнится. 

5. Те, кто был или состоит в браке, чаще возвращают кредит в срок. То же касается, и вдовцов. Есть предположение, что это связано с тем, что эти люди уже привыкли брать на себя ответмственность не только за себя и свои дела, но и за других людей. Вероятно , они более умеют. управлять финансами и планировать траты. 

6. В целом можно говорить о том, что коливчество детей в семей не влияет на возврат кредита в срок, поскольку данные разнятся незначительно. Однако бездетные семьи возвращают кредит в срок - чуть чаще. Вероятно, у семей с детьми случаются внезапные траты и расходые на детей, которые позволяют возвращать кредит в срок. Нулевые значения для семей с пятью детьми, скорее всего, связаны, с маленькой выборкой.

7. Что касается целей кредита, то люди, которые берут кредит на операции с автомобилем и получение образования, реже возврщают кредит в срок. Здесь можно говорить о том, что в большинстве случаев автомобиль - это все-таки дополнительные расходы на использование и обслуживание и при покупке люди ьпросто не учитывают этот момент и дпополнительные расходы. В случае с образованием, вероятно, несколько иная история  - люди предполагают, что образование будет приносить им более высокий доход, однако это случается не всегда. А вот более высокий уровень возврата кредита на недвижимость и свадьбу могут быть связаны с тем, что эти кредиты люди чаще всего берут, рассчитывая на две зарплаты в семье.